<a href="https://colab.research.google.com/github/Jun1233211/CF-Workers-SUB/blob/main/%E2%80%9Ccomfyui%2Bgoogledrive_%E7%BB%88%E6%9E%81%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#步骤 1.  挂载 Drive 并在 Drive 中创建持久化配置


In [3]:

print("\n🚀 正在执行步骤 1/4 ...")

# 1. 导入所有需要的库
from google.colab import drive
import os
import yaml

# 2. 挂载 Google Drive
print("   - 正在挂载 Google Drive...")
drive.mount('/content/drive', force_remount=True)
print("     ✓ Google Drive 已挂载。")

# 3. 定义我们所有文件的根目录 (在 Drive 中)
# 这是我们的“模型仓库”和“配置仓库”
drive_models_base_path = "/content/drive/MyDrive/comfy_models"
os.makedirs(drive_models_base_path, exist_ok=True, ) # 确保这个主目录存在

# 4. 定义正确的 YAML 文件内容
# 使用 'base_path' 是 ComfyUI 的标准、推荐做法，它让文件更整洁。
# 所有其他路径都是相对于 base_path 的。
correct_yaml_content = f"""
comfyui:
    base_path: {drive_models_base_path}
    checkpoints: checkpoints/
    loras: loras/
    vae: vae/
    controlnet: controlnet/
    upscale_models: upscale_models/
    clip_vision: clip_vision/
    gligen: gligen/
    text_encoders: text_encoders/
    unet: unet/
    style_models: style_models/
    embeddings: embeddings/
    diffusers: diffusers/
"""

# 5. 定义配置文件的最终存放位置 (在 Drive 中，而不是临时目录)
config_file_path_in_drive = os.path.join(drive_models_base_path, "extra_model_paths.yaml")

print(f"   - 准备在 Drive 中创建/更新配置文件: {config_file_path_in_drive}")

# 6. 将正确的内容写入文件
with open(config_file_path_in_drive, 'w', encoding='utf-8') as f:
    f.write(correct_yaml_content.strip())

print("     ✓ 持久化配置文件已成功创建/更新。")

# 7. (可选但推荐) 清理 ComfyUI 程序目录中可能存在的旧配置文件，避免冲突
old_config_path = '/content/ComfyUI/extra_model_paths.yaml'
if os.path.exists(old_config_path):
    os.remove(old_config_path)
    print(f"   - 已清理旧的临时配置文件: {old_config_path}")

print("\n--- 配置文件内容检查 (来自 Google Drive) ---")
!cat {config_file_path_in_drive}
print("---------------------------------------------")

print("\n✅ 步骤 1/4 执行完毕。ComfyUI 将自动从您的 Drive 中加载此配置。")



🚀 正在执行步骤 3/4 (优化版)...
   - 正在挂载 Google Drive...
Mounted at /content/drive
     ✓ Google Drive 已挂载。
   - 准备在 Drive 中创建/更新配置文件: /content/drive/MyDrive/comfy_models/extra_model_paths.yaml
     ✓ 持久化配置文件已成功创建/更新。

--- 配置文件内容检查 (来自 Google Drive) ---
comfyui:
    base_path: /content/drive/MyDrive/comfy_models
    checkpoints: checkpoints/
    loras: loras/
    vae: vae/
    controlnet: controlnet/
    upscale_models: upscale_models/
    clip_vision: clip_vision/
    gligen: gligen/
    text_encoders: text_encoders/
    unet: unet/
    style_models: style_models/
    embeddings: embeddings/
    diffusers: diffusers/---------------------------------------------

✅ 步骤 3/4 执行完毕。ComfyUI 将自动从您的 Drive 中加载此配置。


# 步骤2. 检查当前 Torch 版本并安装补充依赖

In [4]:
import os
import sys
import subprocess
import threading
import time
from google.colab import drive
import yaml


print("🚀 步骤 2/5：检测 Colab 自带的 PyTorch 版本...")
# 这一步不会安装或修改 torch，只是打印出当前环境的版本
!python -c "import torch; print(f'✅ Colab 自带的 Torch 版本是: {torch.__version__}')"

print("\n🚀 步骤 3/5：正在安装 ComfyUI 的补充依赖...")
# 我们只安装 ComfyUI 需要、而 Colab 可能没有的包
# 注意：这里没有 --force-reinstall，也没有指定 torch 版本
# Cloudflare 是通过二进制文件安装，而不是 pip
!pip install -q xformers einops safetensors omegaconf transformers huggingface_hub PyYAML

# 克隆 ComfyUI 主程序和插件
comfyui_dir = "/content/ComfyUI"
if not os.path.exists(comfyui_dir):
    !git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git {comfyui_dir}

# 安装 ComfyUI 的 requirements.txt 文件中的依赖
# 这里最有可能出现版本冲突
!pip install -q -r {os.path.join(comfyui_dir, "requirements.txt")}

print("\n✅ 补充依赖安装尝试完成！")

🚀 步骤 1/4：检测 Colab 自带的 PyTorch 版本...
✅ Colab 自带的 Torch 版本是: 2.6.0+cu124

🚀 步骤 2/4：正在安装 ComfyUI 的补充依赖...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 963.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 145.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

# 步骤3.克隆所有指定插件并自动安装全部依赖

In [6]:

import os

print("\n🚀 步骤 4/5   正在准备 ComfyUI 环境...")

# --- 1. 定义 ComfyUI 和插件目录 ---
comfyui_dir = "/content/ComfyUI"
custom_nodes_dir = os.path.join(comfyui_dir, "custom_nodes")
os.makedirs(custom_nodes_dir, exist_ok=True) # 确保插件目录存在

# --- 2. 统一管理所有要安装的插件 (可维护性高) ---
# 只需在此处增删 git 链接即可！
plugins_to_install = {
    "ComfyUI-Manager": "https://github.com/ltdrdata/ComfyUI-Manager.git",
    "Wan2.1": "https://github.com/Wan-Video/Wan2.1.git",
    "ComfyUI-WanVideoWrapper": "https://github.com/kijai/ComfyUI-WanVideoWrapper.git",
    "ComfyUI-Custom-Scripts": "https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git",
    "was-node-suite-comfyui": "https://github.com/WASasquatch/was-node-suite-comfyui.git",
    "rgthree-comfy": "https://github.com/rgthree/rgthree-comfy.git",
    "ComfyUI-Crystools": "https://github.com/crystian/ComfyUI-Crystools.git",
    "ComfyUI-VideoHelperSuite": "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git",
    "ComfyUI_LayerStyle": "https://github.com/chflame163/ComfyUI_LayerStyle.git",
    "ComfyUI-GGUF": "https://github.com/city96/ComfyUI-GGUF.git",
}

# --- 3. 幂等地克隆插件 (可重复运行) ---
print(f"\n   - 正在同步插件到: {custom_nodes_dir}")
for plugin_name, repo_url in plugins_to_install.items():
    plugin_path = os.path.join(custom_nodes_dir, plugin_name)
    if not os.path.exists(plugin_path):
        print(f"     - 克隆新插件: {plugin_name}")
        !git clone --depth 1 {repo_url} {plugin_path}
    else:
        # 如果需要，可以在这里添加 git pull 的逻辑来更新插件
        # print(f"     - 插件已存在，跳过: {plugin_name}")
        pass

print("\n   - 所有插件已同步。")

# --- 4. 自动扫描并安装所有依赖 (健壮且自动) ---
print("\n   - 正在安装所有Python依赖...")

# 首先安装 ComfyUI 主程序的依赖
main_reqs = os.path.join(comfyui_dir, "requirements.txt")
print("     - 安装 ComfyUI 主程序依赖...")
!pip install -q -r {main_reqs}

# 然后自动遍历所有插件目录安装依赖
print("     - 扫描并安装所有插件的依赖...")
for plugin_name in plugins_to_install.keys():
    plugin_path = os.path.join(custom_nodes_dir, plugin_name)
    req_path = os.path.join(plugin_path, "requirements.txt")
    if os.path.exists(req_path):
        print(f"       -> 发现并安装 {plugin_name} 的依赖...")
        !pip install -q -r {req_path}

print("\n✅ 步骤 3 执行完毕：环境、插件和所有依赖已准备就绪。")



🚀 步骤 2 (专业版) - 正在准备 ComfyUI 环境...

   - 正在同步插件到: /content/ComfyUI/custom_nodes
     - 克隆新插件: ComfyUI-Manager
Cloning into '/content/ComfyUI/custom_nodes/ComfyUI-Manager'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 106 (delta 14), reused 47 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (106/106), 961.82 KiB | 3.10 MiB/s, done.
Resolving deltas: 100% (14/14), done.
     - 克隆新插件: Wan2.1
Cloning into '/content/ComfyUI/custom_nodes/Wan2.1'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 70 (delta 8), reused 30 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 9.93 MiB | 9.39 MiB/s, done.
Resolving deltas: 100% (8/8), done.
     - 克隆新插件: ComfyUI-WanVideoWrapper
Cloning into '/content/ComfyUI/custom_nodes/ComfyUI-WanVideoWrapper'...
remote: Enumerating o

# 步骤4.启动 ComfyUI 并使用 Cloudflare 创建公网链接

In [5]:


import torch # 引入 torch 用于检测 GPU

print("\n🚀 步骤 5/5：正在启动 ComfyUI 服务并建立 Cloudflare 隧道...")

# --- 1. 清理可能存在的旧进程，防止端口占用 ---
print("🧹 正在清理可能存在的旧 ComfyUI 和 cloudflared 进程...")
!pkill -f "main.py"
!pkill -f "cloudflared"
time.sleep(3) # 等待进程完全终止

# --- 2. 智能检测环境并构建 ComfyUI 启动命令 ---
use_cpu = not torch.cuda.is_available()

comfyui_run_command = f"python {comfyui_dir}/main.py --listen 0.0.0.0 --port 8188 --preview-method auto"

if use_cpu:
    print("🚦 未检测到 GPU。将以 CPU 模式启动 ComfyUI。")
    comfyui_run_command += " --cpu"
else:
    print("✅ 检测到 GPU！将以标准 GPU 模式启动 ComfyUI。")

# --- 3. 在后台线程中启动 ComfyUI，并捕获日志！---
def run_comfyui(command):
    # 将所有输出重定向到 comfyui.log 文件，这是调试的关键！
    with open('/content/comfyui.log', 'w') as log_file:
        process = subprocess.Popen(command.split(), stdout=log_file, stderr=log_file)

print(f"🚀 正在后台启动 ComfyUI... 命令: `{comfyui_run_command}`")
comfyui_thread = threading.Thread(target=run_comfyui, args=(comfyui_run_command,))
comfyui_thread.start()

print("⏳ 等待 ComfyUI 初始化 (增加到 45 秒，确保有足够时间)...")
time.sleep(45) # 给予更充裕的启动时间

# --- 4. 检查日志文件，提前发现错误 ---
print("\n🔍 检查 ComfyUI 启动日志...")
with open('/content/comfyui.log', 'r') as log_file:
    log_contents = log_file.read()

if "Server started" in log_contents:
    print("✅ ComfyUI 服务似乎已成功启动！")
else:
    print("⚠️ 警告：在日志中未检测到 'Server started'。ComfyUI 可能启动失败。")
    print("   如果隧道无法连接，请务必检查下面的日志全文！")

print("\n--- ComfyUI 日志 (/content/comfyui.log) ---")
!tail -n 20 /content/comfyui.log # 打印日志的最后20行，方便快速查看
print("------------------------------------------")


# --- 5. 下载并启动 Cloudflare 隧道 ---
print("\n tunnel链接🔗...")
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared > /dev/null 2>&1
!chmod +x /usr/local/bin/cloudflared
!cloudflared tunnel --url http://127.0.0.1:8188




🚀 步骤 4/4：正在启动 ComfyUI 服务并建立 Cloudflare 隧道...
🧹 正在清理可能存在的旧 ComfyUI 和 cloudflared 进程...
🚦 未检测到 GPU。将以 CPU 模式启动 ComfyUI。
🚀 正在后台启动 ComfyUI... 命令: `python /content/ComfyUI/main.py --listen 0.0.0.0 --port 8188 --preview-method auto --cpu`
⏳ 等待 ComfyUI 初始化 (增加到 45 秒，确保有足够时间)...

🔍 检查 ComfyUI 启动日志...
⚠️ 警告：在日志中未检测到 'Server started'。ComfyUI 可能启动失败。
   如果隧道无法连接，请务必检查下面的日志全文！

--- ComfyUI 日志 (/content/comfyui.log) ---
  Set XFORMERS_MORE_DETAILS=1 for more details
Set vram state to: DISABLED
Device: cpu
Using sub quadratic optimization for attention, if you have memory or speed issues try using: --use-split-cross-attention
Python version: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
ComfyUI version: 0.3.44
****** User settings have been changed to be stored on the server instead of browser storage. ******
****** For multi-user setups add the --multi-user CLI argument to enable multiple user profiles. ******
ComfyUI frontend version: 1.23.4
[Prompt Server] web root: /usr/local/lib/python3.1


# --- 专用清理单元格：用于重启服务前清理旧进程 ---


In [8]:
# =======================================================
# --- 专用清理单元格：用于重启服务前清理旧进程 ---
# =======================================================
print("🧹 正在清理可能残留的后台进程...")

# -f 标志会匹配整个命令行，确保我们只杀死目标进程
!pkill -f "main.py --listen"  # 终止 ComfyUI 服务
!pkill -f "cloudflared"      # 终止 Cloudflare 隧道

print("✅ 清理完成！现在可以安全地重新运行启动步骤了。")


🧹 正在清理可能残留的后台进程...
✅ 清理完成！现在可以安全地重新运行启动步骤了。


In [ ]:
import os
import sys
import subprocess
import threading
import time
from google.colab import drive
import yaml

# ======================================================================================
# --- 1. 检查当前 Torch 版本并安装补充依赖 ---
# ======================================================================================
print("🚀 步骤 1/4：检测 Colab 自带的 PyTorch 版本...")
# 这一步不会安装或修改 torch，只是打印出当前环境的版本
!python -c "import torch; print(f'✅ Colab 自带的 Torch 版本是: {torch.__version__}')"

print("\n🚀 步骤 2/4：正在安装 ComfyUI 的补充依赖...")
# 我们只安装 ComfyUI 需要、而 Colab 可能没有的包
# 注意：这里没有 --force-reinstall，也没有指定 torch 版本
# Cloudflare 是通过二进制文件安装，而不是 pip
!pip install -q xformers einops safetensors omegaconf transformers huggingface_hub PyYAML

# 克隆 ComfyUI 主程序和插件
comfyui_dir = "/content/ComfyUI"
if not os.path.exists(comfyui_dir):
    !git clone --depth 1 https://github.com/comfyanonymous/ComfyUI.git {comfyui_dir}

# 安装 ComfyUI 的 requirements.txt 文件中的依赖
# 这里最有可能出现版本冲突
!pip install -q -r {os.path.join(comfyui_dir, "requirements.txt")}

print("\n✅ 补充依赖安装尝试完成！")


# ======================================================================================
# --- 2. 挂载 Google Drive 并配置模型路径 ---
# ======================================================================================
print("\n🚀 步骤 3/4：正在挂载 Google Drive 并配置模型路径...")
drive.mount('/content/drive', force_remount=True)

# --- 关键修复：为每种模型类型创建精确的路径指向 ---
# 定义您的模型主文件夹
drive_models_path = "/content/drive/MyDrive/comfy_models"
os.makedirs(drive_models_path, exist_ok=True)

# 创建一个字典，为所有常用模型类型都指定好路径
# 即使文件夹不存在，这个配置也是安全的
path_config = {
    'checkpoints': [os.path.join(drive_models_path, 'checkpoints')],
    'loras': [os.path.join(drive_models_path, 'loras')],
    'vae': [os.path.join(drive_models_path, 'vae')],
    'controlnet': [os.path.join(drive_models_path, 'controlnet')],
    'upscale_models': [os.path.join(drive_models_path, 'upscale_models')],
    'clip_vision': [os.path.join(drive_models_path, 'clip_vision')],
    'gligen': [os.path.join(drive_models_path, 'gligen')],
    'text_encoders': [os.path.join(drive_models_path, 'text_encoders')],
    'unet': [os.path.join(drive_models_path, 'unet')],
    'style_models': [os.path.join(drive_models_path, 'style_models')],
    'embeddings': [os.path.join(drive_models_path, 'embeddings')],
    'diffusers': [os.path.join(drive_models_path, 'diffusers')],
}

# 将配置写入 extra_model_paths.yaml 文件
config_file_path = os.path.join(comfyui_dir, "extra_model_paths.yaml")
with open(config_file_path, 'w') as f:
    yaml.dump(path_config, f)

print(f"✅ 精确的模型路径配置文件已创建！")
print("\n--- 配置文件内容检查 ---")
!cat {config_file_path}
print("--------------------------")


# ======================================================================================
# --- 3. 启动 ComfyUI 并使用 Cloudflare 创建公网链接 ---
# ======================================================================================
print("\n🚀 步骤 4/4：正在尝试启动 ComfyUI 服务并建立 Cloudflare 隧道...")

# 下载并设置 cloudflared
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared

# 定义启动命令
# 将 ComfyUI 启动放到后台运行 (&)，然后启动 cloudflared
cmd_launch = f"python {comfyui_dir}/main.py --listen 0.0.0.0 --port 8188 --preview-method auto &"
comfy_proc = subprocess.Popen(cmd_launch, shell=True)

# 等待 ComfyUI 启动
time.sleep(15)

# 启动 cloudflared 隧道，它会在下方直接打印出链接
!cloudflared tunnel --url http://127.0.0.1:8188